## [Demo] Working with PSANA inside Jupyter lab notebook

This notebook starts a Dask scheduler that serves asa bridge between PSANA and Jupyter Lab. Using the `slurm_magic` package, users can submit a PSANA job and return results to be manipulated within the notebook. 

**Note**: `%srun` is a blocking magic command. 

In [ ]:
print("remove # to clean")
# !rm -f *.json *.log

Load the `slurm_magic` package

In [ ]:
%load_ext slurm_magic
# %squeue -u USERNAME

We proceed to start a scheduler in the login node to be our bridge to share data. 

In [ ]:
from dask.distributed import Client, LocalCluster, Queue, Sub

In [ ]:
scheduler_file = 'scheduler.json'
cluster = LocalCluster(n_workers=1, threads_per_worker=1, host="0.0.0.0")
client = Client(cluster)
client.write_scheduler_file(scheduler_file)

We can now verify our scheduler is up and running. We also created a `scheduler.json` file that can be used to connect as many notebooks as desired.

In [ ]:
import os 

if os.path.isfile(scheduler_file):
    print("Scheduler file with name {} exists!".format(scheduler_file))

In [ ]:
client

Now that client is up and running, there are potentially two ways to submit and gather data from a PSANA job:

1. Using a `Queue` in Dask. 
2. Using Pub/Sub scheme. 

## Queue example

In [ ]:
!python /global/common/software/lcls/psana/lcls2/psana/psana/tests/setup_input_files.py

In [ ]:
%srun -n 3 -A m3384  -C haswell  -q debug  -o test.log python test.py &

In [ ]:
# %squeue -u USERNAME

In [ ]:
q = Queue("psana")

In [ ]:
import numpy as np

results = []
for data in range(q.qsize()):
    results.append(np.array(q.get()))

In [ ]:
results